<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/LSTM_mcfixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [1]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):
    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}

def plot_all_model( sample = None, label=None):
    X = label[:,0]
    Y = label[:,1]
    img = sample[:,:].squeeze().cpu().numpy()
    plt.imshow(img, cmap=plt.cm.gray_r)
    predres = 1000

    s = [.001 for x in range(predres)]
    
    assert len(s) == predres
    c = ['red' for x in range(predres)]
    assert len(c) == predres
    plt.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)


def getXYs(model=None,loader_disp = None):

    outputs = torch.zeros(100,1000,2).cuda()
    with torch.no_grad():
      h0 = None
      c0 = None
      X = []
      Y = []
      for samples,labels in loader_disp:
        for i in range(38):
          optimizer.zero_grad()
          if i == 0:
            out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i)
            outputs[:,i*25:(i+1)*25,:] = out.detach().clone()
          else:
            out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i,o = o,output=output)
            outputs[:,i*25:(i+1)*25,:] = out.detach().clone()

        return samples, outputs

          #X.extend(out.detach()[0,:,0].tolist())
          #Y.extend(out.detach()[0,:,1].tolist())

          
        

def plot_all( sample = None, labels = None):
    img = sample[:,:].squeeze().cpu().numpy()
    plt.imshow(img, cmap=plt.cm.gray_r)
    X = labels[:,0]
    Y = labels[:,1]
    s = [.001 for x in range(numpoints)]
    c = ['red' for x in range(numpoints)]
    plt.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
    
class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,dataset, model):
        #model.setBatchSize(batch_size = 1)
        if model != None:
          model = model.eval()
          loader_disp = data.DataLoader(
            dataset, 
            batch_size=100,
            num_workers=2)
          samples, outputs = getXYs(model=model,loader_disp = loader_disp)
          for i in range(100):
            plt.subplot(10,10,i+1)
            plot_all_model(samples[i,:,:],outputs[i,:,:])
            plt.axis('off')

        else:
          for i in range(100):
            sample, labels = dataset[i]
            plt.subplot(10,10,i+1)
            plot_all(sample = sample,labels = labels)
            plt.axis('off')

        plt.savefig(title,dpi=450)
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [2]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
dataset = DonutDataset(length = 100*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=2)


x0_y0_r_xrf_yrf torch.Size([2000, 1]) torch.Size([2000, 1]) torch.Size([2000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [20]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 512
        self.num_layers = 2
        self.seq_len = 25
        self.full_len = 1000
        self.lm_len = int(self.full_len/self.seq_len)
        self.mem_fac = 5
        self.mem_index = 0

        self.longtermMem_lstm = None
        
        self.lstm_cell = nn.LSTM(self.hidden_size, self.hidden_size,self.num_layers)

        self.inject_image_lstm =  nn.Sequential(
            nn.Linear(2+1024+6*self.hidden_size,self.hidden_size),
            #nn.Dropout(),
            nn.Tanh()
        )

        self.fc_out = nn.Sequential(
            nn.Linear(self.hidden_size,2),
            nn.Sigmoid()
        )
        

        self.alpha_lstm = nn.Sequential(
            nn.Linear(self.hidden_size,self.seq_len),
            nn.Softmax()
        )
        
        self.alpha_long_lstm_r1 = nn.Sequential(
            nn.Linear(self.hidden_size,int((self.seq_len/self.mem_fac)*self.lm_len)),
            nn.Softmax()
        )

        self.alpha_long_lstm_r2 = nn.Sequential(
            nn.Linear(self.hidden_size,int((self.seq_len/self.mem_fac)*self.lm_len)),
            nn.Softmax()
        )
        
        self.alpha_long_lstm_r3 = nn.Sequential(
            nn.Linear(self.hidden_size,int((self.seq_len/self.mem_fac)*self.lm_len)),
            nn.Softmax()
        )

        self.alpha_long_lstm_r4 = nn.Sequential(
            nn.Linear(self.hidden_size,int((self.seq_len/self.mem_fac)*self.lm_len)),
            nn.Softmax()
        )
        
        self.embs = None #= torch.empty((self.batch_size, 1000, self.hidden_size)).cuda()


    def forward(self, features,p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        assert features != None
        assert len(features.shape) == 3
        self.track +=1
        batch_size = features.shape[0]
        
        p0 = torch.flatten(p0[:,:2],start_dim=1)
        
        if o == None:
          out = p0
          self.longtermMem_lstm = torch.zeros(batch_size,int((self.seq_len/self.mem_fac)*self.lm_len),self.hidden_size).cuda()
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          self.mem_index = 0
        else:
          out = o
        
        if h0 == None and c0 == None:
          h0 = torch.zeros(self.num_layers,batch_size,self.hidden_size).cuda()
          c0 = torch.zeros(self.num_layers,batch_size,self.hidden_size).cuda()

        hidden_state = h0
        cell_state = c0
        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        
        output = None

        for t in range(self.seq_len):

            
          if output == None:
            output = torch.zeros(batch_size,self.hidden_size).cuda()
            outesread1_lstm = torch.zeros(batch_size,self.hidden_size).cuda()
            outesreadl1_lstm = torch.zeros(batch_size,self.hidden_size).cuda()
            outesreadl2_lstm = torch.zeros(batch_size,self.hidden_size).cuda()
            outesreadl3_lstm = torch.zeros(batch_size,self.hidden_size).cuda()
            outesreadl4_lstm = torch.zeros(batch_size,self.hidden_size).cuda()
          else:
            a1_lstm = self.alpha_lstm(output.view(batch_size,self.hidden_size))
            outesreada1_lstm = self.embs.clone()*a1_lstm.unsqueeze(2)#clone
            outesread1_lstm = torch.sum(outesreada1_lstm,dim=1)
            
            al1_lstm = self.alpha_long_lstm_r1(output.view(batch_size,self.hidden_size))
            outesreada1_lstm = self.longtermMem_lstm.clone()*al1_lstm.unsqueeze(2)#clone
            outesreadl1_lstm = torch.sum(outesreada1_lstm,dim=1)
            
            al2_lstm = self.alpha_long_lstm_r2(output.view(batch_size,self.hidden_size))
            outesreada2_lstm = self.longtermMem_lstm.clone()*al2_lstm.unsqueeze(2)#clone
            outesreadl2_lstm = torch.sum(outesreada2_lstm,dim=1)
            
            al3_lstm = self.alpha_long_lstm_r3(output.view(batch_size,self.hidden_size))
            outesreada3_lstm = self.longtermMem_lstm.clone()*al3_lstm.unsqueeze(2)#clone
            outesreadl3_lstm = torch.sum(outesreada3_lstm,dim=1)
            
            al4_lstm = self.alpha_long_lstm_r4(output.view(batch_size,self.hidden_size))
            outesreada4_lstm = self.longtermMem_lstm.clone()*al4_lstm.unsqueeze(2)#clone
            outesreadl4_lstm = torch.sum(outesreada4_lstm,dim=1)
            

            
          combin_lstm = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),outesread1_lstm,outesreadl1_lstm,outesreadl2_lstm,outesreadl3_lstm,outesreadl4_lstm],dim=1)
          combin_lstm = self.inject_image_lstm(combin_lstm)
          
          self.embs[:, t, :] = output.view(batch_size,self.hidden_size).clone()#.detach()
          
          output, (hidden_state, cell_state) = self.lstm_cell(combin_lstm.unsqueeze(0), (hidden_state, cell_state))
          if t%self.mem_fac == 0:
            self.longtermMem_lstm[:,self.mem_index,:] = output.view(batch_size,self.hidden_size).detach()
            self.mem_index+=1
          

          out = self.fc_out(output.view(batch_size,self.hidden_size))
          out = out*32.0
          outputs[:, t, :] = out
            
        return outputs, hidden_state.detach(), cell_state.detach(), out.detach(),output.detach()#, captions


In [21]:
model = DecoderRNN().cuda()

In [22]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [23]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [29]:
model = model.train()
for seql in range(3,38,4):
  #seql = 19
  for epoch in range (4):
    loss = None
    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print(loss)

a
b


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 1
tensor(35.7087, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(30.5459, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(28.1397, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(22.6423, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(19.9387, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(15.6052, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(11.1572, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(7.5264, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(5.3828, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(4.8236, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(4.9272, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(6.2538, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(6.8162, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 1
tensor(6.4392, device='cuda:0', grad_fn=<M

In [8]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001


In [9]:

model = model.train()
#for seql in range(15,20,4):
seql = 38
for epoch in range (15):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 38
tensor(6.8810, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(6.2873, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.7074, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.1764, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(5.0846, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.1647, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(4.2134, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(12.1028, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.5335, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.3034, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.3715, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(7.3361, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.0153, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(2.0995, device='cuda:0', gr

KeyboardInterrupt: ignored

In [11]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [12]:

model = model.train()
#for seql in range(15,20,4):
seql = 38
for epoch in range (40):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 38
tensor(1.5352, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.3936, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.3485, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.2763, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.3192, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.1847, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.2072, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.2242, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.3319, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.2832, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.1752, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.3028, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.2268, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.1796, device='cuda:0', gra

In [19]:
model = model.eval()
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

with torch.no_grad():
  for epoch in range (3):

    mini_batch = 100
    test_dataset = DonutDataset(length = 100*10)
    loader_test = data.DataLoader(
        test_dataset, 
        batch_size=mini_batch,
        sampler=RandomSampler(data_source=test_dataset),
        num_workers=2)

    loss = None
    for x,y in loader_test:
      h0 = None
      c0 = None
      seql = 38
      loss = torch.zeros(seql)
      for i in range(seql):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        
        #print(x.shape,y.shape)
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
        loss[i] = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        print(loss[i])
        writer.add_scalar("Loss/val", loss[i],i, epoch)
        

      #print(y[:,0,:2])
      print('epoch',epoch)
      print('test loss', torch.mean(loss))

x0_y0_r_xrf_yrf torch.Size([1000, 1]) torch.Size([1000, 1]) torch.Size([1000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tens

KeyboardInterrupt: ignored

In [ ]:
writer.flush()
writer.close()


In [ ]:
#%load_ext tensorboard

In [ ]:
#!kill 3285
#%tensorboard --logdir runs

In [15]:
pred_dataset = DonutDataset(length = 100)

x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [ ]:
!nvidia-smi

In [18]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  lstmpredictions.png


In [17]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('trainlstmpredictions.png',dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  trainlstmpredictions.png


In [ ]:
model = model.eval()
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

with torch.no_grad():
  for epoch in range (3):

    mini_batch = 100
    test_dataset = DonutDataset(length = 100*10)
    loader_test = data.DataLoader(
        test_dataset, 
        batch_size=mini_batch,
        sampler=RandomSampler(data_source=test_dataset),
        num_workers=2)

    loss = None
    for x,y in loader_test:
      h0 = None
      c0 = None
      seql = 38
      loss = torch.zeros(seql)
      for i in range(seql):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        
        #print(x.shape,y.shape)
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
        loss[i] = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        print(loss[i])
        writer.add_scalar("Loss/val", loss[i],i, epoch)
        

      #print(y[:,0,:2])
      print('epoch',epoch)
      print('test loss', torch.mean(loss))

In [ ]:
for name, param in model.named_parameters():
  print(name)